In [47]:
import pandas as pd
import numpy as np
import requests
import json


In [48]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [99]:
df = pd.read_csv('adresy-Warszawa.csv', sep=';')
df

,dzielnica,nazwa_peln,nazwa_skr,numer,x,y,id_instancji,kod
0,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,101,7.494187e+06,5.789198e+06,1272121,01-355
1,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,102,7.494238e+06,5.789209e+06,1272123,01-355
2,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,103,7.494222e+06,5.789175e+06,1272107,01-355
3,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,104,7.494244e+06,5.789179e+06,1272108,01-355
4,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,105,7.494265e+06,5.789184e+06,1272110,01-355
...,...,...,...,...,...,...,...,...
112615,Żoliborz,ulica Zygmunta Krasińskiego,ul. Z. Krasińskiego,67,7.497536e+06,5.791730e+06,1435331,01-755
112616,Żoliborz,ulica Zygmunta Krasińskiego,ul. Z. Krasińskiego,69,7.497481e+06,5.791741e+06,1435334,01-755
112617,Żoliborz,ulica Zygmunta Krasińskiego,ul. Z. Krasińskiego,7,7.499482e+06,5.792954e+06,1434278,01-530
112618,Żoliborz,ulica Zygmunta Krasińskiego,ul. Z. Krasińskiego,8,7.499286e+06,5.792946e+06,1434266,01-601


In [100]:
data = df.drop_duplicates('kod').reset_index(drop=True)
data.head()

,dzielnica,nazwa_peln,nazwa_skr,numer,x,y,id_instancji,kod
0,Bemowo,Osiedle Przyjaźń,os. Przyjaźń,101,7.494187e+06,5.789198e+06,1272121,01-355
1,Bemowo,Plac Kasztelański,pl. Kasztelański,1,7.494292e+06,5.788650e+06,1271876,01-362
2,Bemowo,ulica Antoniego Bolesława Dobrowolskiego,ul. A. B. Dobrowolskiego,1,7.492704e+06,5.791632e+06,1273498,01-483
3,Bemowo,ulica Arki Bożka,ul. A. Bożka,1,7.494352e+06,5.789961e+06,1275040,01-464
4,Bemowo,ulica Alberta Einsteina,ul. A. Einsteina,1,7.492734e+06,5.791192e+06,1273347,01-480


In [101]:
data = data.drop(columns=['nazwa_peln', 'numer', 'x', 'y', 'id_instancji'])
data.head()

,dzielnica,nazwa_skr,kod
0,Bemowo,os. Przyjaźń,01-355
1,Bemowo,pl. Kasztelański,01-362
2,Bemowo,ul. A. B. Dobrowolskiego,01-483
3,Bemowo,ul. A. Bożka,01-464
4,Bemowo,ul. A. Einsteina,01-480


In [102]:
data = data.reindex(columns=['kod', 'dzielnica', 'nazwa_skr'])
data = data.rename(columns={'kod': 'Postcode', 'dzielnica': 'Borough', 'nazwa_skr': 'Street'})
data.head()

,Postcode,Borough,Street
0,01-355,Bemowo,os. Przyjaźń
1,01-362,Bemowo,pl. Kasztelański
2,01-483,Bemowo,ul. A. B. Dobrowolskiego
3,01-464,Bemowo,ul. A. Bożka
4,01-480,Bemowo,ul. A. Einsteina


In [104]:
data.groupby('Borough').size()

Borough
Bemowo            106
Białołęka         184
Bielany           194
Mokotów           355
Ochota            192
Praga-Południe    378
Praga-Północ      144
Rembertów         114
Targówek          267
Ursus             5  
Ursynów           96 
Wawer             444
Wesoła            2  
Wilanów           39 
Wola              313
Włochy            206
Śródmieście       551
Żoliborz          150
dtype: int64

In [159]:
data['Latitude'] = np.nan
data['Longitude'] = np.nan
data.head(10)

,Postcode,Borough,Street,Latitude,Longitude
0,01-355,Bemowo,os. Przyjaźń,NaN,NaN
1,01-362,Bemowo,pl. Kasztelański,NaN,NaN
2,01-483,Bemowo,ul. A. B. Dobrowolskiego,NaN,NaN
3,01-464,Bemowo,ul. A. Bożka,NaN,NaN
4,01-480,Bemowo,ul. A. Einsteina,NaN,NaN
5,01-473,Bemowo,ul. A. Kocjana,NaN,NaN
6,01-391,Bemowo,ul. A. Krzywoń,NaN,NaN
7,01-494,Bemowo,ul. A. Sołtana,NaN,NaN
8,01-318,Bemowo,ul. A. Świętochowskiego,NaN,NaN
9,01-497,Bemowo,ul. Afrodyty,NaN,NaN


In [176]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


address = 'Pałac Kultury, Warszawa'

geolocator = Nominatim(user_agent="warsaw")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of center of Warsaw are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of center of Warsaw are 52.2317641, 21.005799675616117.


In [185]:
import time

%time
# Getting the latitude and longitude of an address

for i in range(10):
    postcode = data.at[i, 'Postcode']
    borough = data.at[i, 'Borough']
    street = data.at[i, 'Street']
    address = '{}, {}, Polska'.format(postcode, borough)
    
    geolocator = Nominatim(user_agent='warsaw_neighborhoods')
    #location = geolocator.geocode(address)
    
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=10, error_wait_seconds=3)
    location = geocode(address)
    print(address)
    print(location)
    print(location.latitude)


    data.at[i, 'Latitude'] = location.latitude
    data.at[i, 'Longitude'] = location.longitude
    time.sleep(1)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
01-355, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-355, Polska
52.23763509588161
01-362, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-362, Polska
52.231731438109776
01-483, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-483, Polska
52.257719615798855
01-464, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-464, Polska
52.242481470544696
01-480, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, Polska
52.2389738
01-473, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-473, Polska
52.25080441182424
01-391, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-391, Polska
52.22788142519653
01-494, Bemowo, Polska
Osiedle Leśne, Bemowo, Warszawa, województwo mazowieckie, 01-494, Polska
52.25463297147664
01-318, Bemowo, Polska
Bemowo, Warszawa, województwo mazowieckie, 01-318, Polska
52.23417542086964
01-497, Bemowo, Polska
Bemowo, Warszawa, w

In [178]:
type(data.at[0, 'Postcode'])

str

In [179]:
data.head(5)

,Postcode,Borough,Street,Latitude,Longitude
0,01-355,Bemowo,os. Przyjaźń,52.237635,20.916247
1,01-362,Bemowo,pl. Kasztelański,52.231731,20.915943
2,01-483,Bemowo,ul. A. B. Dobrowolskiego,52.257720,20.892278
3,01-464,Bemowo,ul. A. Bożka,52.242481,20.915951
4,01-480,Bemowo,ul. A. Einsteina,52.238974,20.913288


In [180]:


import geocoder # import geocoder

# initialize your variable to None


for i in range(5):
    lat_lng_coords = None
    postcode = data.at[i, 'Postcode']
    borough = data.at[i, 'Borough']
    street = data.at[i, 'Street']
    address = '{}, {}, Warszawa, Polska'.format(postcode, borough)
    print(address)
    
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google(address)
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    print(latitude)
    print(longitude)

01-355, Bemowo, Warszawa, Polska


KeyboardInterrupt: 